# Zenodo API code for getting GitHub Repos 

### Setup  

Ensure you are in the `coding-smart-github` conda environment and have the following python packages in your environment:
  * requests    

#### Github Authentication

Create an access token via Zenodo Applications settings [here](https://zenodo.org/account/settings/applications/tokens/) and create a file called `zenodoconfig.cfg` with the following content:

    [ACCESS]
    token = <your-access-token>
    
Ensure you've pasted in your token, but leave `[ACCESS]` and `token =` .

In [114]:
import pandas as pd
#import numpy as np 
import os 
from os import path
import configparser 
import requests
import csv 

In [10]:
import requests
r = requests.get("https://zenodo.org/api/deposit/depositions")
r.status_code
# 403
r.json()

{'status': 403, 'message': 'Permission denied.'}

In [11]:
# set up zenodo access token

config = configparser.ConfigParser()
config.read('../zenodoconfig.cfg')
config.sections()

access_token = config['ACCESS']['token']

r = requests.get('https://zenodo.org/api/deposit/depositions',
                  params={'access_token': access_token})

print(r)

r.status_code
# 200

r.json()
# []



<Response [200]>


[{'created': '2015-06-25T15:04:19+00:00',
  'modified': '2020-01-24T19:27:28.405139+00:00',
  'id': 12271,
  'conceptrecid': '592847',
  'doi': '10.5281/zenodo.12271',
  'conceptdoi': '',
  'doi_url': 'https://doi.org/10.5281/zenodo.12271',
  'metadata': {'title': 'peramagroon: seed',
   'doi': '10.5281/zenodo.12271',
   'publication_date': '2014-10-16',
   'description': '<p>This is the beginning of my reproducible research / open data / science journey.</p>\n\n<p>I&#39;ll use this repo to document my progress in analysing and publishing results from data collected during fieldwork carried out during my masters (MSc) thesis. The original thesis was on the vegetation of Peramagroon Mountain in Iraqi Kurdistan, and was completed in August 2013.</p>\n\n<p>It will contain my data, data cleaning/tidying and analysis scripts, figures, and manuscripts. I&#39;m also aiming to make everything reproducible and openly accessible, so I&#39;ll also detail these factors and include notes on how I d

In [67]:
records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software AND creators.affiliation:Cambridge'
res = requests.get(records_api_url, params={'q': search_query, 'size': 25, 'page': 1})

In [68]:
res.json()

[{'created': '2023-10-05T08:04:04.220306+00:00',
  'modified': '2023-10-05T14:27:13.088874+00:00',
  'id': 8409512,
  'conceptrecid': '1435860',
  'doi': '10.5281/zenodo.8409512',
  'conceptdoi': '10.5281/zenodo.1435860',
  'doi_url': 'https://doi.org/10.5281/zenodo.8409512',
  'metadata': {'title': 'AMUSE: the Astrophysical Multipurpose Software Environment',
   'doi': '10.5281/zenodo.8409512',
   'publication_date': '2023-10-05',
   'description': 'A Python framework to combine existing astrophysical simulation codes in numerical experiments. With AMUSE you can simulate objects such as star clusters, proto-planetary disks and galaxies.',
   'access_right': 'open',
   'creators': [{'name': 'Portegies Zwart, Simon',
     'affiliation': 'Leiden University',
     'orcid': '0000-0001-5839-0302'},
    {'name': 'van Elteren, Arjen', 'affiliation': 'Leiden University'},
    {'name': 'Pelupessy, Inti', 'affiliation': 'Netherlands eScience Center'},
    {'name': 'McMillan, Steve', 'affiliation

In [ ]:
# Field details here: https://help.zenodo.org/guides/search/  


In [80]:
records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software'
res = requests.get(records_api_url, params={'q': search_query, 'size': 3, 'page': 1})

In [81]:
res.json()

[{'created': '2022-11-22T15:29:21.125713+00:00',
  'modified': '2022-11-23T02:26:37.566894+00:00',
  'id': 7347926,
  'conceptrecid': '3908559',
  'doi': '10.5281/zenodo.7347926',
  'conceptdoi': '10.5281/zenodo.3908559',
  'doi_url': 'https://doi.org/10.5281/zenodo.7347926',
  'metadata': {'title': 'ultralytics/yolov5: v7.0 - YOLOv5 SOTA Realtime Instance Segmentation',
   'doi': '10.5281/zenodo.7347926',
   'publication_date': '2022-11-22',
   'description': '<p>&lt;div align="center"&gt;\n&lt;a align="center" href="https://ultralytics.com/yolov5" target="_blank"&gt;\n&lt;img width="850" src="https://github.com/ultralytics/assets/blob/master/yolov5/v70/splash.png"&gt;&lt;/a&gt;\n&lt;/div&gt;\n&lt;br&gt;</p>\n<p>Our new YOLOv5 v7.0 instance segmentation models are the fastest and most accurate in the world, beating all current <a href="https://paperswithcode.com/sota/real-time-instance-segmentation-on-mscoco">SOTA benchmarks</a>. We\'ve made them super simple to train, validate and de

In [93]:
res.json()[0].get('created')

'2022-11-22T15:29:21.125713+00:00'

In [94]:
res.json()[0].get('metadata')

{'title': 'ultralytics/yolov5: v7.0 - YOLOv5 SOTA Realtime Instance Segmentation',
 'doi': '10.5281/zenodo.7347926',
 'publication_date': '2022-11-22',
 'description': '<p>&lt;div align="center"&gt;\n&lt;a align="center" href="https://ultralytics.com/yolov5" target="_blank"&gt;\n&lt;img width="850" src="https://github.com/ultralytics/assets/blob/master/yolov5/v70/splash.png"&gt;&lt;/a&gt;\n&lt;/div&gt;\n&lt;br&gt;</p>\n<p>Our new YOLOv5 v7.0 instance segmentation models are the fastest and most accurate in the world, beating all current <a href="https://paperswithcode.com/sota/real-time-instance-segmentation-on-mscoco">SOTA benchmarks</a>. We\'ve made them super simple to train, validate and deploy. See full details in our <a href="https://github.com/ultralytics/yolov5/releases/v7.0">Release Notes</a> and visit our <a href="https://github.com/ultralytics/yolov5/blob/master/segment/tutorial.ipynb">YOLOv5 Segmentation Colab Notebook</a> for quickstart tutorials.</p>\n<p>&lt;div align="ce

In [95]:
res.json()[0].get('metadata').get('title')

'ultralytics/yolov5: v7.0 - YOLOv5 SOTA Realtime Instance Segmentation'

In [96]:
res.json()[0].get('metadata').keys()

dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'related_identifiers', 'version', 'license', 'imprint_publisher', 'upload_type', 'prereserve_doi'])

In [107]:
res.json()[0].get('metadata').get('related_identifiers')

[{'identifier': 'https://github.com/ultralytics/yolov5/tree/v7.0',
  'relation': 'isSupplementTo',
  'scheme': 'url'}]

In [111]:
res.json()[0].get('metadata').get('related_identifiers')

[{'identifier': 'https://github.com/ultralytics/yolov5/tree/v7.0',
  'relation': 'isSupplementTo',
  'scheme': 'url'}]

In [116]:
content = pd.DataFrame.from_dict(res.json())
 # pulling the json query output into a dataframe 
    # can then begin to pull data out of metadata column... 
    # ... where it's got related_identifiers including github.com

In [118]:
print(content)

                            created                          modified  \
0  2022-11-22T15:29:21.125713+00:00  2022-11-23T02:26:37.566894+00:00   
1  2023-09-15T14:02:38.609718+00:00  2023-09-15T14:26:57.370604+00:00   
2  2023-10-05T08:49:40.550242+00:00  2023-10-05T14:27:13.161473+00:00   

        id conceptrecid                     doi              conceptdoi  \
0  7347926      3908559  10.5281/zenodo.7347926  10.5281/zenodo.3908559   
1  8349181       596518  10.5281/zenodo.8349181   10.5281/zenodo.596518   
2  8409685      1243862  10.5281/zenodo.8409685  10.5281/zenodo.1243862   

                                  doi_url  \
0  https://doi.org/10.5281/zenodo.7347926   
1  https://doi.org/10.5281/zenodo.8349181   
2  https://doi.org/10.5281/zenodo.8409685   

                                            metadata  \
0  {'title': 'ultralytics/yolov5: v7.0 - YOLOv5 S...   
1  {'title': 'python-pillow/Pillow: 10.0.1', 'doi...   
2  {'title': 'Scientific colour maps', 'doi': '10...   



In [144]:
content['related_info'] = content[['metadata']].apply(lambda x: [x.get('related_identifiers') for x in x])

In [145]:
content

,created,modified,id,conceptrecid,doi,conceptdoi,doi_url,metadata,title,links,record_id,owner,files,state,submitted,related_info
0,2022-11-22T15:29:21.125713+00:00,2022-11-23T02:26:37.566894+00:00,7347926,3908559,10.5281/zenodo.7347926,10.5281/zenodo.3908559,https://doi.org/10.5281/zenodo.7347926,{'title': 'ultralytics/yolov5: v7.0 - YOLOv5 S...,ultralytics/yolov5: v7.0 - YOLOv5 SOTA Realtim...,{'self': 'https://zenodo.org/api/records/73479...,7347926,64193,[{'id': '6d9a4918-34ef-47ed-a472-e11bb6edcdcf'...,done,True,[{'identifier': 'https://github.com/ultralytic...
1,2023-09-15T14:02:38.609718+00:00,2023-09-15T14:26:57.370604+00:00,8349181,596518,10.5281/zenodo.8349181,10.5281/zenodo.596518,https://doi.org/10.5281/zenodo.8349181,"{'title': 'python-pillow/Pillow: 10.0.1', 'doi...",python-pillow/Pillow: 10.0.1,{'self': 'https://zenodo.org/api/records/83491...,8349181,17549,[{'id': '453c7390-aa5c-4ca8-b727-d0f11c837828'...,done,True,[{'identifier': 'https://github.com/python-pil...
2,2023-10-05T08:49:40.550242+00:00,2023-10-05T14:27:13.161473+00:00,8409685,1243862,10.5281/zenodo.8409685,10.5281/zenodo.1243862,https://doi.org/10.5281/zenodo.8409685,"{'title': 'Scientific colour maps', 'doi': '10...",Scientific colour maps,{'self': 'https://zenodo.org/api/records/84096...,8409685,43498,[{'id': 'df9ec0f1-6ad8-46d2-8485-5e93eb5d8593'...,done,True,"[{'identifier': '10.5194/gmd-2017-328', 'relat..."


In [147]:
content['related_info_gh'] = content[['related_info']].applymap(lambda x: [x.get('identifier') for x in x])

In [148]:
content

,created,modified,id,conceptrecid,doi,conceptdoi,doi_url,metadata,title,links,record_id,owner,files,state,submitted,related_info,related_info_gh
0,2022-11-22T15:29:21.125713+00:00,2022-11-23T02:26:37.566894+00:00,7347926,3908559,10.5281/zenodo.7347926,10.5281/zenodo.3908559,https://doi.org/10.5281/zenodo.7347926,{'title': 'ultralytics/yolov5: v7.0 - YOLOv5 S...,ultralytics/yolov5: v7.0 - YOLOv5 SOTA Realtim...,{'self': 'https://zenodo.org/api/records/73479...,7347926,64193,[{'id': '6d9a4918-34ef-47ed-a472-e11bb6edcdcf'...,done,True,[{'identifier': 'https://github.com/ultralytic...,[https://github.com/ultralytics/yolov5/tree/v7.0]
1,2023-09-15T14:02:38.609718+00:00,2023-09-15T14:26:57.370604+00:00,8349181,596518,10.5281/zenodo.8349181,10.5281/zenodo.596518,https://doi.org/10.5281/zenodo.8349181,"{'title': 'python-pillow/Pillow: 10.0.1', 'doi...",python-pillow/Pillow: 10.0.1,{'self': 'https://zenodo.org/api/records/83491...,8349181,17549,[{'id': '453c7390-aa5c-4ca8-b727-d0f11c837828'...,done,True,[{'identifier': 'https://github.com/python-pil...,[https://github.com/python-pillow/Pillow/tree/...
2,2023-10-05T08:49:40.550242+00:00,2023-10-05T14:27:13.161473+00:00,8409685,1243862,10.5281/zenodo.8409685,10.5281/zenodo.1243862,https://doi.org/10.5281/zenodo.8409685,"{'title': 'Scientific colour maps', 'doi': '10...",Scientific colour maps,{'self': 'https://zenodo.org/api/records/84096...,8409685,43498,[{'id': 'df9ec0f1-6ad8-46d2-8485-5e93eb5d8593'...,done,True,"[{'identifier': '10.5194/gmd-2017-328', 'relat...",[10.5194/gmd-2017-328]


In [149]:
#### START HERE #####

#### continue by pulling the github.com/*/* content out into a file. 


In [119]:
search_query = "software"
r = requests.get("https://zenodo.org/api/records", params={'type': 'software'})

identifiers = []

if 'hits' in r.json():
    still_iterating = True
else:
    still_iterating = False

while still_iterating and (len(identifiers) < 30):
    if 'hits' in r.json():
        for hit in r.json()['hits']['hits']:
#            print(hit['id'])
#        print(hit['metadata']['title'])
            identifiers.append(hit['id'])
            
        if 'next' in r.json()['links']:
            r = requests.get(r.json()['links']['next'])
        else: 
            still_iterating = False
    else:
        still_iterating = False
        
print(identifiers)

[]


In [4]:
# Get GitHub links from a list of Zenodo IDs

f = open('results.csv', 'w')
writer = csv.writer(f)

header = ['Zenodo ID', 'GitHub Related Link', 'Relation Type']
writer.writerow(header)

for record_id in identifiers:
    r = requests.get("https://zenodo.org/api/records/{}".format(record_id))

    for id in r.json()['metadata']['related_identifiers']:
        if ("github.com" in id['identifier']):
            row = []
            row.append(record_id)
            row.append(id['identifier'])
            row.append(id['relation'])
            writer.writerow(row)

f.close()

In [22]:
# get zenodo records with type software  
# via: https://github.com/npch/reporunner/blob/main/notebooks/zenodo.ipynb

#search_query = "software"
#r = requests.get("https://zenodo.org/api/records", params={'type': 'software'})  
  # this comes from the format w/in 'records' entity here: https://pythonhosted.org/zenodo/api/records.html  

This search just pulls software records from the api ^  

In [5]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [9]:
r.headers

{'server': 'nginx', 'date': 'Mon, 16 Oct 2023 15:14:32 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'x-ratelimit-limit': '60', 'x-ratelimit-remaining': '58', 'x-ratelimit-reset': '1697469332', 'retry-after': '59', 'permissions-policy': 'interest-cohort=()', 'x-frame-options': 'sameorigin', 'x-xss-protection': '1; mode=block', 'x-content-type-options': 'nosniff', 'content-security-policy': "default-src 'self' fonts.googleapis.com *.gstatic.com data: 'unsafe-inline' 'unsafe-eval' blob: zenodo-broker.web.cern.ch zenodo-broker-qa.web.cern.ch maxcdn.bootstrapcdn.com cdnjs.cloudflare.com ajax.googleapis.com webanalytics.web.cern.ch", 'strict-transport-security': 'max-age=31556926; includeSubDomains, max-age=15768000', 'referrer-policy': 'strict-origin-when-cross-origin', 'access-control-allow-origin': '*', 'access-control-expose-headers': 'Content-Type, ETag, Link, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset', 'set-cooki

In [20]:
r.url

'https://zenodo.org/api/records?type=software'

In [6]:
r.json()

[{'created': '2022-02-07T15:56:41.044307+00:00',
  'modified': '2022-03-01T17:10:55.581497+00:00',
  'id': 5997682,
  'conceptrecid': '5997681',
  'doi': '10.5281/zenodo.5997682',
  'conceptdoi': '10.5281/zenodo.5997681',
  'doi_url': 'https://doi.org/10.5281/zenodo.5997682',
  'metadata': {'title': 'Calycomyza artemisivora Eiseman & Lonsdale 2018, spec. nov.',
   'doi': '10.5281/zenodo.5997682',
   'publication_date': '2018-09-14',
   'description': '<i>Calycomyza artemisivora</i> spec. nov.   <p>(Figs. 12, 14, 102, 257&ndash;261)</p>  <p> <b>Holotype: OKLAHOMA:</b> Payne Co., Mehan, 36.014339&deg; N, 96.996744&deg; W, 10.v.2016, em. 31.v.2016, M.W. Palmer, ex <i>Artemisia ludoviciana</i>, #CSE2526, CNC653998 (1&male;).</p>   <p> <b>Paratype: OKLAHOMA:</b> same collection as holotype, CNC 653997 (1&male;).</p>   <p> <b>Etymology.</b> The specific epithet refers to the host plant, <i>Artemisia</i> L.</p>   <p> <b>Host.</b> Asteraceae: <i>Artemisia ludoviciana</i> Nutt.</p>   <p> <b>Lea

In [121]:
search_query = "software"
r = requests.get("https://zenodo.org/api/records", params={'type': 'software'})

identifiers = []

if 'hits' in r.json():
    still_iterating = True
else:
    still_iterating = False

while still_iterating and (len(identifiers) < 30):
    if 'hits' in r.json():
        for hit in r.json()['hits']['hits']:
#            print(hit['id'])
#        print(hit['metadata']['title'])
            identifiers.append(hit['id'])
            
        if 'next' in r.json()['links']:
            r = requests.get(r.json()['links']['next'])
        else: 
            still_iterating = False
    else:
        still_iterating = False
        
print(identifiers)

[]


In [130]:
test

[{'created': '2023-06-12T15:47:37.186009+00:00',
  'modified': '2023-06-13T02:27:04.462573+00:00',
  'id': 8028697,
  'conceptrecid': '8028696',
  'doi': '10.5281/zenodo.8028697',
  'conceptdoi': '10.5281/zenodo.8028696',
  'doi_url': 'https://doi.org/10.5281/zenodo.8028697',
  'metadata': {'title': 'Microbiome enhanced addiction - simulations',
   'doi': '10.5281/zenodo.8028697',
   'publication_date': '2023-06-12',
   'description': '<p>Two simulation scripts for host-microbiome modeling</p>',
   'access_right': 'open',
   'creators': [{'name': 'Ohad Lewin-Epstein',
     'affiliation': None,
     'orcid': '0000-0002-8636-9006'},
    {'name': 'Yanabah Jaques', 'affiliation': None},
    {'name': 'Marcus Feldman', 'affiliation': None},
    {'name': 'Daniela Kaufer', 'affiliation': None},
    {'name': 'Lilach Hadany', 'affiliation': None}],
   'license': 'cc-by-4.0',
   'imprint_publisher': 'Zenodo',
   'upload_type': 'software',
   'prereserve_doi': {'doi': '10.5281/zenodo.8028697', 're

In [135]:
test = pd.DataFrame.from_dict(test)


In [139]:
test

,created,modified,id,conceptrecid,doi,conceptdoi,doi_url,metadata,title,links,record_id,owner,files,state,submitted,related_info
0,2023-06-12T15:47:37.186009+00:00,2023-06-13T02:27:04.462573+00:00,8028697,8028696,10.5281/zenodo.8028697,10.5281/zenodo.8028696,https://doi.org/10.5281/zenodo.8028697,{'title': 'Microbiome enhanced addiction - sim...,Microbiome enhanced addiction - simulations,{'self': 'https://zenodo.org/api/records/80286...,8028697,26150,[{'id': 'ed298b26-71bf-4036-9aa8-bdafb6abf048'...,done,True,None
1,2023-06-17T09:35:32.309659+00:00,2023-06-17T14:26:44.831046+00:00,8050209,8050208,10.5281/zenodo.8050209,10.5281/zenodo.8050208,https://doi.org/10.5281/zenodo.8050209,{'title': 'Combined use of 87Sr/86Sr and carbo...,Combined use of 87Sr/86Sr and carbon isotopes ...,{'self': 'https://zenodo.org/api/records/80502...,8050209,216696,[{'id': '409541ea-1462-4da5-8440-503b46bce638'...,done,True,None
2,2023-07-14T07:42:58.988437+00:00,2023-07-14T14:26:43.911836+00:00,8146268,8146267,10.5281/zenodo.8146268,10.5281/zenodo.8146267,https://doi.org/10.5281/zenodo.8146268,{'title': 'DIALEKTOLOGIYA SOHASIDA ANDIJON SHE...,DIALEKTOLOGIYA SOHASIDA ANDIJON SHEVALARINING ...,{'self': 'https://zenodo.org/api/records/81462...,8146268,213214,[{'id': '16e100da-ec74-45e1-8742-0b8a9c209b02'...,done,True,None
3,2023-07-14T19:03:08.056483+00:00,2023-07-14T19:14:19.626533+00:00,8148304,8148303,10.5281/zenodo.8148304,10.5281/zenodo.8148303,https://doi.org/10.5281/zenodo.8148304,{'title': 'Şekil 16. Stigmaeus glabrisetus (Dö...,Şekil 16. Stigmaeus glabrisetus (Dötonimf). Al...,{'self': 'https://zenodo.org/api/records/81483...,8148304,1161,[{'id': '2bef39d1-6b9e-48e5-8cf7-bea8202d5c5d'...,done,True,"[{'identifier': '10.47121/acarolstud.696796', ..."
4,2023-07-24T12:53:58.908401+00:00,2023-07-25T02:26:50.577080+00:00,8177677,8177676,10.1029/2022GL101184,NaN,https://doi.org/10.1029/2022GL101184,{'title': 'Stronger Response to the Aerosol In...,Stronger Response to the Aerosol Indirect Effe...,{'self': 'https://zenodo.org/api/records/81776...,8177677,549383,[{'id': 'f38ebf72-2a19-4448-a793-03cc6c7f5baf'...,done,True,None
5,2023-04-18T11:29:37.567897+00:00,2023-04-19T02:26:44.688465+00:00,7840632,7840631,10.5281/zenodo.7840632,10.5281/zenodo.7840631,https://doi.org/10.5281/zenodo.7840632,{'title': 'Andrena (Ulandrena) graciliata Wood...,"Andrena (Ulandrena) graciliata Wood 2023, spec...",{'self': 'https://zenodo.org/api/records/78406...,7840632,1161,[{'id': 'a4cd7fcc-372d-4e38-913d-5c3997128b92'...,done,True,"[{'identifier': '10.11646/zootaxa.5266.1.1', '..."
6,2023-07-13T06:27:23.803783+00:00,2023-07-13T14:26:45.598286+00:00,8141500,8141499,10.11591/ijeecs.v31.i2.pp785-793,NaN,https://doi.org/10.11591/ijeecs.v31.i2.pp785-793,{'title': 'Unorthodox technique in sensing wit...,Unorthodox technique in sensing with the metam...,{'self': 'https://zenodo.org/api/records/81415...,8141500,148978,[{'id': '6a3648a1-78c8-4376-ac8a-341b38408c5b'...,done,True,None
7,2023-07-27T12:42:45.999925+00:00,2023-07-27T14:26:48.776733+00:00,8189468,8189467,10.5281/zenodo.8189468,10.5281/zenodo.8189467,https://doi.org/10.5281/zenodo.8189468,"{'title': 'Filsafat Bahasa dan Teori Bahasa', ...",Filsafat Bahasa dan Teori Bahasa,{'self': 'https://zenodo.org/api/records/81894...,8189468,52048,[{'id': '715af0ac-886c-423b-9986-c499e86f5620'...,done,True,None
8,2023-04-19T21:18:10.553430+00:00,2023-04-20T02:26:40.140563+00:00,7847487,7847486,10.5281/zenodo.7847487,10.5281/zenodo.7847486,https://doi.org/10.5281/zenodo.7847487,{'title': 'AMIR TEMUR BUYUK DAVLAT ARBOBI VA S...,AMIR TEMUR BUYUK DAVLAT ARBOBI VA SARKARDA,{'self': 'https://zenodo.org/api/records/78474...,7847487,285581,[{'id': '42502505-f5c5-4f1b-8fc1-28bfa9cea5ba'...,done,True,None
9,2023-02-16T13:18:56.722477+00:00,2023-02-17T02:26:43.721320+00:00,7646499,7646498,10.5281/zenodo.7646499,10.5281/zenodo.7646498,https://doi.org/10.5281/zenodo.7646499,"{'title': 'SciExpeM - 10.5281/zenodo.7646499',...",SciExpeM - 10.5281/

TypeError: 'NoneType' object is not iterable